<a href="https://colab.research.google.com/github/mjmousavi97/Computer-Vision/blob/main/projects/pro-004/src/GoogleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import wandb

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Transforms for training and validation
data_transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
wandb.init(
    project="Inception-5flowers", config={
        "learning_rate": 0.001,
        "epochs": 50,
        "batch_size": 16,
        "architecture": "InceptionV1",
        "pretrained": True,
        "input_size":224
    }
)

config = wandb.config

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 1


wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true&ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammadjavad-mousavi97 (mohammadjavad-mousavi97-arak-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [10]:

train_dir = "/content/drive/MyDrive/Colab Notebooks/flowers/train"
val_dir = "/content/drive/MyDrive/Colab Notebooks/flowers/val"

train_dataset = datasets.ImageFolder(train_dir, transform=data_transform['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transform['val'])

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)


In [11]:
from torchvision.models import GoogLeNet_Weights

model = models.googlenet(weights=GoogLeNet_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 5)

for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

wandb.watch(model, log="all", log_freq=10)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


100%|██████████| 49.7M/49.7M [00:00<00:00, 129MB/s]


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

In [13]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10, log_every=10):
    device = next(model.parameters()).device

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for step, (images, labels) in enumerate(train_loader):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            bs = labels.size(0)
            running_loss += loss.item() * bs
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += bs

            if (step + 1) % log_every == 0:
                train_loss_step = running_loss / max(1, total)
                train_acc_step = correct / max(1, total)
                wandb.log(
                    {
                        "epoch": epoch + 1,
                        "step": epoch * len(train_loader) + (step + 1),
                        "train/loss": train_loss_step,
                        "train/acc": train_acc_step,
                        "lr": optimizer.param_groups[0]["lr"],
                    }
                )

        train_loss = running_loss / max(1, total)
        train_acc = correct / max(1, total)

        model.eval()
        val_loss_sum = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                outputs = model(images)
                loss = criterion(outputs, labels)

                bs = labels.size(0)
                val_loss_sum += loss.item() * bs
                preds = outputs.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += bs

        val_loss = val_loss_sum / max(1, val_total)
        val_acc = val_correct / max(1, val_total)

        wandb.log(
            {
                "epoch": epoch + 1,
                "train/loss_epoch": train_loss,
                "train/acc_epoch": train_acc,
                "val/loss": val_loss,
                "val/acc": val_acc,
            }
        )

        print(
            f"Epoch {epoch+1}/{num_epochs} | "
            f"train_loss={train_loss:.4f} train_acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} val_acc={val_acc:.4f}"
        )

    print("Training complete!")


In [15]:
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=config.epochs)

Epoch 1/50 | train_loss=0.9029 train_acc=0.7061 | val_loss=0.5588 val_acc=0.8259
Epoch 2/50 | train_loss=0.5980 train_acc=0.7964 | val_loss=0.5009 val_acc=0.8309
Epoch 3/50 | train_loss=0.5106 train_acc=0.8246 | val_loss=0.4528 val_acc=0.8556
Epoch 4/50 | train_loss=0.4818 train_acc=0.8283 | val_loss=0.4413 val_acc=0.8566
Epoch 5/50 | train_loss=0.4646 train_acc=0.8393 | val_loss=0.3882 val_acc=0.8754
Epoch 6/50 | train_loss=0.4556 train_acc=0.8386 | val_loss=0.3859 val_acc=0.8675
Epoch 7/50 | train_loss=0.4426 train_acc=0.8416 | val_loss=0.3728 val_acc=0.8783
Epoch 8/50 | train_loss=0.4318 train_acc=0.8473 | val_loss=0.3816 val_acc=0.8734
Epoch 9/50 | train_loss=0.4313 train_acc=0.8423 | val_loss=0.3732 val_acc=0.8714
Epoch 10/50 | train_loss=0.4158 train_acc=0.8508 | val_loss=0.3618 val_acc=0.8704
Epoch 11/50 | train_loss=0.4217 train_acc=0.8466 | val_loss=0.3586 val_acc=0.8813
Epoch 12/50 | train_loss=0.4162 train_acc=0.8538 | val_loss=0.3627 val_acc=0.8724
Epoch 13/50 | train_loss=